# Benchmarking some sorting algorithms
The idea of this notebook is to show how one can compare algorithms via `@benchmark` in Julia. The algorithms we compare are just simple sorting algorithms and the implementations are very naive. The point of this notebook are not the algorithms themselves.

In [1]:
using Pkg; Pkg.add("BenchmarkTools")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2


In [2]:
using BenchmarkTools

In [6]:
function is_sorted(v::Vector{Int})
    for i in 1:(length(v)-1)
        if v[i+1] < v[i]
            return false
        end
    end
    return true
end

is_sorted (generic function with 1 method)

In [7]:
is_sorted([1,2,3,4])

true

In [8]:
is_sorted([1,2,3,2])

false

## Permute randomly
We can sort a vector by just permuting it randomly until it is sorted.

In [9]:
using Pkg; Pkg.add("Random")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [9a3f8284] + Random
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [10]:
using Random

In [12]:
function shuffle_sort(v::Vector{Int})
    result = copy(v)
    while !is_sorted(result)
        result = copy(v)
        shuffle!(result)
    end
    return result
end

shuffle_sort (generic function with 1 method)

Check that it works:

In [13]:
v = [1,2,4,3]
shuffle_sort(v)

4-element Vector{Int64}:
 1
 2
 3
 4

We benchmark on a bunch of random vectors:

In [16]:
@benchmark shuffle_sort(rand(Int, 10))

BenchmarkTools.Trial: 15 samples with 1 evaluation.
 Range (min … max):   23.237 ms …    1.405 s  ┊ GC (min … max): 2.95% … 2.09%
 Time  (median):     223.008 ms               ┊ GC (median):    2.06%
 Time  (mean ± σ):   336.297 ms ± 355.082 ms  ┊ GC (mean ± σ):  2.14% ± 0.31%

  ▁▁ ▁██  ▁   █ ▁     ▁ ▁       ▁                             ▁  
  ██▁███▁▁█▁▁▁█▁█▁▁▁▁▁█▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  23.2 ms          Histogram: frequency by time           1.4 s <

 Memory estimate: 29.71 MiB, allocs estimate: 216314.

## Ascending sort
(Maybe the name is wrong) Slightly better: Always start at the beginning. If there is a neighboring pair in the wrong order, permute it.

In [23]:
function ascending_sort(v::Vector{Int})
    result = copy(v)
    i = 1
    while i < length(result)
        if result[i+1] < result[i]
            tmp = result[i]
            result[i] = result[i+1]
            result[i+1] = tmp
            if i > 1
                i -= 1
            end
        else
            i += 1
        end
    end
    return result
end

ascending_sort (generic function with 1 method)

In [24]:
is_sorted(ascending_sort(rand(Int,10)))

true

In [25]:
@benchmark ascending_sort(rand(Int, 10))

BenchmarkTools.Trial: 10000 samples with 755 evaluations.
 Range (min … max):  166.314 ns …  1.632 μs  ┊ GC (min … max): 0.00% … 83.72%
 Time  (median):     173.154 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   187.050 ns ± 82.817 ns  ┊ GC (mean ± σ):  2.87% ±  5.83%

  ▄██▇▇▅▄▃▃▂▂▃▄▃▂▂▁▁▁▁▁▁▁▁▁▁                                   ▂
  ███████████████████████████▇██▇▇▇▇█▇████████▇▇▆▆▅▆▆▅▆▅▆▃▅▅▄▅ █
  166 ns        Histogram: log(frequency) by time       281 ns <

 Memory estimate: 288 bytes, allocs estimate: 2.

We see that the performance is already much more solid than the one of `shuffle_sort`. Can you guess why this also uses far less memory?

## Builtin algorithm
We use Julias builtin implementation `sort`.

In [27]:
is_sorted(sort(rand(Int,10)))

true

In [28]:
@benchmark sort(rand(Int,10))

BenchmarkTools.Trial: 10000 samples with 780 evaluations.
 Range (min … max):  161.597 ns …  1.712 μs  ┊ GC (min … max): 0.00% … 82.60%
 Time  (median):     166.809 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   179.896 ns ± 81.641 ns  ┊ GC (mean ± σ):  2.96% ±  5.87%

  ▇█▇▆▄▃▂▂▂▂▄▃▂▁▁▁▁ ▁▁  ▁                                      ▂
  █████████████████████████▇▇▆▇▅▇▇███▇▇█▇▆▅▅▅▄▄▄▅▄▄▃▅▃▅▅▅▅▅▄▄▅ █
  162 ns        Histogram: log(frequency) by time       287 ns <

 Memory estimate: 288 bytes, allocs estimate: 2.

This is similar to `ascending_sort`. So we compare on something larger.

In [34]:
@benchmark sort(rand(Int,100))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.698 μs … 119.450 μs  ┊ GC (min … max): 0.00% … 97.47%
 Time  (median):     1.835 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.971 μs ±   1.836 μs  ┊ GC (mean ± σ):  1.59% ±  1.69%

      ▅█▆▁                                                     
  ▁▁▃▇████▅▃▁▁▁▁▂▂▃▃▃▃▃▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.7 μs          Histogram: frequency by time         2.9 μs <

 Memory estimate: 1.75 KiB, allocs estimate: 2.

In [33]:
@benchmark ascending_sort(rand(Int, 100))

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  4.757 μs … 197.792 μs  ┊ GC (min … max): 0.00% … 97.27%
 Time  (median):     5.343 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.495 μs ±   2.859 μs  ┊ GC (mean ± σ):  0.69% ±  1.37%

         ▂▄▆▇█▇▄▄▂▂▁                                           
  ▁▁▁▂▃▅▇████████████▇▆▆▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  4.76 μs         Histogram: frequency by time        7.35 μs <

 Memory estimate: 1.75 KiB, allocs estimate: 2.